## XGBoost

XGBoost is a gradient boosting algorithm built on decision trees. 
It works fast even with large datasets because it evaluates best split for each feature in parallel. 
Also it uses a column block structure in memory. Column-wise storage is helpful because all instances of a feature 
are stored in a contiguous memory. It also supports distributed computing and can handle missing values. 

In [1]:
# download covertype dataset
import os
import urllib.request


DOWNLOAD_ROOT = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/"
COVER_PATH = os.path.join("datasets", "covertype")

def fetch_covertype_data(cover_path=COVER_PATH):
    if not os.path.isdir(cover_path):
        os.makedirs(cover_path)
    for filename in ("covtype.data.gz", "covtype.info"):
        url = DOWNLOAD_ROOT + filename
        urllib.request.urlretrieve(url, os.path.join(cover_path, filename))

fetch_covertype_data()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>

In [ ]:
# load the data
import pandas as pd

def load_covertype_data(cover_path=COVER_PATH):
    csv_path = os.path.join(cover_path, "covtype.data.gz")
    return pd.read_csv(csv_path, header=None)

cover = load_covertype_data()

# keep header names as per the info file

cover.columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                    'Horizontal_Distance_To_Fire_Points'] + \
                    ['Wilderness_Area_{}'.format(i) for i in range(4)] + \
                    ['Soil_Type_{}'.format(i) for i in range(40)] + \
                    ['Cover_Type']

# print the first five rows
print(cover.head())

# split into X and Y
X = cover.drop(['Cover_Type'], axis=1)
y = cover['Cover_Type']

In [ ]:
# data exploration
import matplotlib.pyplot as plt
# check if there is missing data
print("Number of missing features - {}".format(X.isnull().sum()))

# check all the unique data types
print("Unique datatypes {}".format(X.dtypes.unique()))

# everything is int64, so no need to convert

# check the distribution of the data
X.hist(bins=50, figsize=(40,30))
plt.show()

y.hist(bins=50, figsize=(40,30))
plt.show()


# get statistical summary of the data
print("Summary of the data")
print(X.iloc[:,0:9].describe())

# plot a histogram of the wilderness area 
plt.figure(figsize=(10, 5))
plt.bar(range(4), X.iloc[:,10:14].sum())
plt.xticks(range(4), ['Rawah', 'Neota', 'Comanche Peak', 'Cache la Poudre'])
plt.ylabel('Count')
plt.show()

# plot a histogram of the soil type
plt.figure(figsize=(10, 5))
plt.bar(range(40), X.iloc[:,14:54].sum())
plt.xticks(range(40), range(1,41))
plt.ylabel('Count')
plt.show()


In [ ]:
# check if there are outliers in the data
# plot box plot for each feature separately
plt.figure(figsize=(20, 10))
for idx, col in enumerate(X.iloc[:,0:9].columns):
    plt.subplot(3,3,idx+1)
    plt.boxplot(X[col])
    plt.title(col)
plt.show()

# Calculate Z-score for each feature
from scipy import stats
z = np.abs(stats.zscore(X.iloc[:,0:9]))

# count the number of outliers for each feature
print("Number of outliers for each feature")
print(np.sum(z>3, axis=0))


In [ ]:
# check if some soil types occur more than others
print("Soil type counts")
print(X.iloc[:,14:54].sum(axis=0))


soil_type_14 exists only in three instances out of 500k. 
There seem to be many outliers that are outside the median +/- 1.5*IQR. Should we remove those features?
Hillshades seem to be correlated with aspect and slope. 
Horizontal distance to highways seem to be correlated with elevation. 

In [ ]:
# find the correlation between the features using scipy
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler

def normalize(X):
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        return X_scaled

# calculate spearman correlation
corr, _ = spearmanr(X.iloc[:,0:9])
print("Spearman correlation")
print(corr)

# plot the correlation matrix 
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
cmap = cm.get_cmap('jet', 30)
cax = ax1.imshow(corr, interpolation="nearest", cmap=cmap)
# ax1.grid(True)
plt.title('Spearman Correlation')
labels=['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
        'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
        'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
ticks = np.arange(len(labels))
ax1.set_xticks(ticks)
ax1.set_yticks(ticks)
ax1.set_xticklabels(labels,fontsize=6)
print(ax1.get_xticklabels())
# put the xtick at 90 degrees
plt.setp(ax1.get_xticklabels(), rotation=90)
ax1.set_yticklabels(labels,fontsize=6)
# Add colorbar, make sure to specify tick locations to match desired ticklabels
fig.colorbar(cax, ticks=[-0.5,-0.25,0,0.25,0.5,0.75,1])
plt.show()

# normalize the numerical features into a different dataframe
X_scaled = X.copy()
X_scaled.iloc[:,0:9] = normalize(X.iloc[:,0:9])

# y is from 1 to 7, encode it to 0 to 6
y_encoded = y - 1
# create a mapping from y to cover type
cover_type = {0:'Spruce/Fir', 1:'Lodgepole Pine', 2:'Ponderosa Pine', 3:'Cottonwood/Willow',
              4:'Aspen', 5:'Douglas-fir', 6:'Krummholz'}


# print the statistical summary of the first 9 features
print("Summary of the data")
print(X_scaled.iloc[:,0:9].describe())


In [ ]:
# split the data into train, test
X_train_full, X_test, y_train_full, y_test = train_test_split(X_scaled, y_encoded, test_size=0.8,
                                                              random_state=42, stratify=y_encoded)

# split the data into train, validation
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2,
                                                  random_state=42, stratify=y_train_full)
# train a xgboost classifier

import xgboost as xgb
from sklearn.metrics import accuracy_score

# optimize the parameters using GridSearchCV
from sklearn.model_selection import GridSearchCV

# learning rate in XGBoost controls how much weight is given to previous trees
learning_rate = [0.001, 0.1, 0.5, 1]

# fit the model for different learning rates and find the best one

# also use early stopping to find the optimal number of trees
best_accuracy = 0
best_xgb_clf = None

fig, ax = plt.subplots()
for lr in learning_rate:
    xgb_clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators=1000,
                                learning_rate=lr, early_stopping_rounds=10, random_state=42, eval_metric='mlogloss')
    xgb_clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = xgb_clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print('learning rate: {}, accuracy: {}'.format(lr, accuracy))

    # plot the validation error in the same plot for different learning rates
    results = xgb_clf.evals_result()
    epochs = len(results['validation_0']['mlogloss'])
    x_axis = range(0, epochs)
    ax.plot(x_axis, results['validation_0']['mlogloss'], label='lr={}'.format(lr))
    ax.legend()
    # store the best classifier
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_xgb_clf = xgb_clf

plt.ylabel('Validation error')
plt.title('XGBoost Validation error')
plt.show()


In [ ]:
# print the best accuracy and number of trees
print('best accuracy: {}'.format(best_accuracy))
print('best number of trees: {}'.format(best_xgb_clf.best_iteration))

# sum up the feature importance of all widerness areas and soil types separately
wilderness_importance = np.sum(best_xgb_clf.feature_importances_[10:14])
soil_importance = np.sum(best_xgb_clf.feature_importances_[14:54])

# get the feature importance of the first 9 features
feature_importance = best_xgb_clf.feature_importances_[0:9]

# append the wilderness and soil importance to the feature importance
feature_importance = np.append(feature_importance, wilderness_importance)
feature_importance = np.append(feature_importance, soil_importance)

# plot the feature importance for first 9 features, wilderness and soil types

# first get fig and axis
fig, ax = plt.subplots()

# set size of fig
fig.set_size_inches(10, 5)

# plot the bar chart
plt.bar(range(11), feature_importance)
plt.xticks(range(11), ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Wilderness', 'Soil_Type'])

# rotate the xticks by 90 degrees
plt.setp(ax.get_xticklabels(), rotation=90)    
plt.ylabel('Feature Importance')
plt.show()


Soil_Type is a very important feature. Wilderness and Elevation also seem to be important. 

In [ ]:
# calculate test accuracy
y_pred = best_xgb_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print('test accuracy: {}'.format(test_accuracy))


- Test accuracy is pretty good and is close to validation accuracy. 